In [3]:
import time
import requests
from bs4 import BeautifulSoup
import re
from zipfile import ZipFile
import glob
import os
import fitz
import openai
import pandas as pd
from tenacity import retry, stop_after_attempt, wait_random_exponential

In [3]:
def get_page_content(url):
    headers = {
        'User-Agent': 'my_crawler (brandon.loorits@ut.ee) / for_study_purpose',
    } # Määrame enda päringu päise, et oleks teada, kes päringuid veebileheküljele teeb
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.text
    else:
        print(f'Päring lehele {url} ebaõnnestus. Staatuskood: {response.status_code}')
        return None

In [4]:
main_url = 'https://nasdaqbaltic.com/statistics/et/shares'
shares = []

# Kogume kokku kõik url-id
print(f'Külastan lehte: {main_url}')
time.sleep(5)  # Viiteaeg, et ei ummistaks lehte
page_content = get_page_content(main_url)
if page_content:
    soup = BeautifulSoup(page_content, 'html.parser')
    table = soup.find('table') 
    if table:
        links = [a['href'] for a in table.find_all('a', href=True, class_="text16 compname") if a['href'].startswith('/statistics/et/instrument')]
        shares.extend(links)
    else:
        print('Ei leitud tabelit".')
else:
    print('Algse lehe külastamine ebaõnnestus.')
    

Külastan lehte: https://nasdaqbaltic.com/statistics/et/shares


In [5]:
print(shares)

['/statistics/et/instrument/LT0000128092/trading', '/statistics/et/instrument/LT0000102337/trading', '/statistics/et/instrument/EE3100034653/trading', '/statistics/et/instrument/LT0000127466/trading', '/statistics/et/instrument/EE3100007857/trading', '/statistics/et/instrument/LV0000101806/trading', '/statistics/et/instrument/EE3100016965/trading', '/statistics/et/instrument/EE3100127242/trading', '/statistics/et/instrument/EE3100137985/trading', '/statistics/et/instrument/LT0000102030/trading', '/statistics/et/instrument/EE3100004250/trading', '/statistics/et/instrument/EE3100082306/trading', '/statistics/et/instrument/LV0000101863/trading', '/statistics/et/instrument/LT0000115768/trading', '/statistics/et/instrument/EE3100149394/trading', '/statistics/et/instrument/LT0000111650/trading', '/statistics/et/instrument/EE3100102203/trading', '/statistics/et/instrument/EE3100098328/trading', '/statistics/et/instrument/EE3100039496/trading', '/statistics/et/instrument/LT0000131872/trading',

In [6]:
page_content = get_page_content('https://nasdaqbaltic.com/statistics/et/instrument/LT0000128092/trading')
soup = BeautifulSoup(page_content, 'html.parser')
link_element = soup.find('a', string="Aruanded")
link = link_element.get('href')
print(link)

/statistics/et/instrument/LT0000128092/reports?date=2024-04-24


In [7]:
# Külastame leitud linke
reports=[]
for link in shares:
    absolute_link = f'https://nasdaqbaltic.com{link}'
    print(f'Külastan lehte: {absolute_link}')
    time.sleep(5)  # Ootame 5 sekundit enne järgmise päringu tegemist, et mitte ummistada lehekülge
    page_content = get_page_content(absolute_link)
    if page_content:
        soup = BeautifulSoup(page_content, 'html.parser')
        link_element = soup.find('a', string="Aruanded")
        reports.append(link_element.get('href'))
    else:
        print(f'Lehe külastamine ebaõnnestus: {absolute_link}')
print(len(reports))


Külastan lehte: https://nasdaqbaltic.com/statistics/et/instrument/LT0000128092/trading
Külastan lehte: https://nasdaqbaltic.com/statistics/et/instrument/LT0000102337/trading
Külastan lehte: https://nasdaqbaltic.com/statistics/et/instrument/EE3100034653/trading
Külastan lehte: https://nasdaqbaltic.com/statistics/et/instrument/LT0000127466/trading
Külastan lehte: https://nasdaqbaltic.com/statistics/et/instrument/EE3100007857/trading
Külastan lehte: https://nasdaqbaltic.com/statistics/et/instrument/LV0000101806/trading
Külastan lehte: https://nasdaqbaltic.com/statistics/et/instrument/EE3100016965/trading
Külastan lehte: https://nasdaqbaltic.com/statistics/et/instrument/EE3100127242/trading
Külastan lehte: https://nasdaqbaltic.com/statistics/et/instrument/EE3100137985/trading
Külastan lehte: https://nasdaqbaltic.com/statistics/et/instrument/LT0000102030/trading
Külastan lehte: https://nasdaqbaltic.com/statistics/et/instrument/EE3100004250/trading
Külastan lehte: https://nasdaqbaltic.com/st

In [8]:
print(reports[5])

/statistics/et/instrument/LV0000101806/reports?date=2024-04-24


In [9]:
base_link =f'https://nasdaqbaltic.com'
xhtml_nums = {
    'ako':('pf57','pf38_1'),
    'apg':('pf1','pf2c'),
    'ign':('pfa3','pf136'),
    'kne1':('pfa3','pf100'),
    'pzv':('pf57','pf70'),
    'rsu':('pf1','pf34'),
    'sab':('pf1','pf4c'),
    'vlp':('pf1','pf4b')
}
exclusion_list = xhtml_nums.keys()
# print(exclusion_list)
# Loome kaustad, kui need ei eksisteeri
os.makedirs('aruanded/aastaaruanded', exist_ok=True)
os.makedirs('alusfailid', exist_ok=True)

for report in reports:
    absolute_link = f'{base_link}{report}'
    print(f'Külastan lehte: {absolute_link}')
    time.sleep(5)  # Ootame 5 sekundit enne järgmise päringu tegemist, et mitte ummistada lehekülge
    page_content = get_page_content(absolute_link)
    soup = BeautifulSoup(page_content, 'html.parser')
    table = soup.find('tbody')

    pdf_link = None
    zip_link = None
    esg_link = None

    for row in table.find_all('tr'):
        #print(row)
        links = row.find_all('a')
        hrefs = [link.get('href') for link in links]
        #print(hrefs)
        for href in hrefs:
            # print(href)
            if 'ar' in href.split('/')[-1] and href.endswith('.pdf') and href.split('/')[4] not in exclusion_list:
                if href.split('/')[4] == 'dgr' and href.endswith('ias.pdf'): # erand ühele aruandele kuna sellel olemas ka lühendatud versioon, mida me ei vaja
                    pdf_link = href
                    break
                else:
                    pdf_link = href
            elif 'ar' in href.split('/')[-1] and href.endswith('.zip') and not pdf_link:
                zip_link = href
            if 'esg' in href.split('/')[-1]:
                esg_link = href
        if pdf_link or zip_link:
            break

    # Kontrollime, kas aastaaruanne on olemas
    if pdf_link:
        # Kontrollime kõigepealt, kas ESG aruanne on olemas samale aastale
        if esg_link:
            esg_link_full = f'{base_link}{esg_link}'
            print(f'ESG link: {esg_link_full}')
            esg_response = requests.get(esg_link_full)
            if esg_response:
                match = re.search(r'/reports/([^_/]+/[^_/]+)_', esg_link_full).group(1).replace('/', '_')
                print(f'Kirjutame esg {match} pdfi maha')
                with open(f'aruanded/aastaaruanded/esg_{match}.pdf', 'wb') as file:
                    file.write(esg_response.content)
        else:# Kui ei ole, siis võtame href atribuudi väärtuse ja kirjutame aastaaruande kausta
            rep_link = f'{base_link}{pdf_link}'
            print(f'REP link: {rep_link}')
                    
            rep_response = requests.get(rep_link)
            if rep_response:
                match = re.search(r'/reports/([^_/]+/[^_/]+)_', rep_link).group(1).replace('/', '_')
                print(f'Kirjutame aastaaruande pdfi {match} maha')
                with open(f'aruanded/aastaaruanded/rep_{match}.pdf', 'wb') as file:
                    file.write(rep_response.content)
    elif zip_link:
        # Võtame href atribuudi väärtuse
        rep_link = f'{base_link}{zip_link}'
        print(f'REP link: {rep_link}')
                
        rep_response = requests.get(rep_link)
        if rep_response:
            match = re.search(r'/reports/([^_/]+/[^_/]+)_', rep_link).group(1).replace('/', '_')
            print(f'Kirjutame aastaaruande zipi {match} maha')
            with open(f'alusfailid/rep_{match}.zip', 'wb') as file:
                file.write(rep_response.content)


dict_keys(['ako', 'apg', 'ign', 'kne1', 'pzv', 'rsu', 'sab', 'vlp'])
Külastan lehte: https://nasdaqbaltic.com/statistics/et/instrument/LT0000128092/reports?date=2024-04-24
REP link: https://nasdaqbaltic.com/market/upload/reports/ako/2023_ar_en_eur_con_ias_esef.zip
Kirjutame aastaaruande zipi ako_2023 maha
Külastan lehte: https://nasdaqbaltic.com/statistics/et/instrument/LT0000102337/reports?date=2024-04-24
REP link: https://nasdaqbaltic.com/market/upload/reports/apg/2023_ar_en_eur_con_ias_esef.zip
Kirjutame aastaaruande zipi apg_2023 maha
Külastan lehte: https://nasdaqbaltic.com/statistics/et/instrument/EE3100034653/reports?date=2024-04-24
REP link: https://nasdaqbaltic.com/market/upload/reports/arc/2023_ar_et_eur_con_00.pdf
Kirjutame aastaaruande pdfi arc_2023 maha
Külastan lehte: https://nasdaqbaltic.com/statistics/et/instrument/LT0000127466/reports?date=2024-04-24
REP link: https://nasdaqbaltic.com/market/upload/reports/aug/2022_ar_en_eur_con_ias.pdf
Kirjutame aastaaruande pdfi aug_

In [10]:
def leia_aruanne(zip_path, target_dir_rep):
    # Kontrollime, kas väljundkaust on juba olemas
    if not os.path.exists(target_dir_rep):
        os.makedirs(target_dir_rep)
    name = zip_path.split('\\')[1].replace('zip','xhtml')
    with ZipFile(zip_path, 'r') as zip_ref:
        for file_name in zip_ref.namelist():
            # Kontrollime, kas faili laiend on .xhtml 
            if file_name.endswith('.xhtml'):
                with zip_ref.open(file_name) as file:
                    content = file.read()
                    
                new_file_path = os.path.join(target_dir_rep, os.path.basename(name))
                print(new_file_path)
                # Kirjutame sisu uude faili sihtkaustas
                with open(new_file_path, 'wb') as new_file:
                    new_file.write(content)
                print(f"Fail {os.path.basename(name)} on kirjutatud kausta {target_dir_rep}.")

In [11]:
target_dir_rep = 'aruanded/aruandedXHTML'

zip_files = glob.glob('alusfailid/*.zip')

for zip in zip_files:
    leia_aruanne(zip,target_dir_rep)

aruanded/aruandedXHTML\rep_ako_2023.xhtml
Fail rep_ako_2023.xhtml on kirjutatud kausta aruanded/aruandedXHTML.
aruanded/aruandedXHTML\rep_apg_2023.xhtml
Fail rep_apg_2023.xhtml on kirjutatud kausta aruanded/aruandedXHTML.
aruanded/aruandedXHTML\rep_grg_2023.xhtml
Fail rep_grg_2023.xhtml on kirjutatud kausta aruanded/aruandedXHTML.
aruanded/aruandedXHTML\rep_ign_2023.xhtml
Fail rep_ign_2023.xhtml on kirjutatud kausta aruanded/aruandedXHTML.
aruanded/aruandedXHTML\rep_kne1_2023.xhtml
Fail rep_kne1_2023.xhtml on kirjutatud kausta aruanded/aruandedXHTML.
aruanded/aruandedXHTML\rep_pzv_2023.xhtml
Fail rep_pzv_2023.xhtml on kirjutatud kausta aruanded/aruandedXHTML.
aruanded/aruandedXHTML\rep_rsu_2023.xhtml
Fail rep_rsu_2023.xhtml on kirjutatud kausta aruanded/aruandedXHTML.
aruanded/aruandedXHTML\rep_sab_2023.xhtml
Fail rep_sab_2023.xhtml on kirjutatud kausta aruanded/aruandedXHTML.
aruanded/aruandedXHTML\rep_vlp_2023.xhtml
Fail rep_vlp_2023.xhtml on kirjutatud kausta aruanded/aruandedXHTML.

In [14]:
def extract_text_from_xhtml(xhtml_files, id_ranges):
    for xhtml_file in xhtml_files:
        file_name = xhtml_file.split('\\')[-1].split('.')[0].split('_')[1]
        

        if file_name in id_ranges:
            start_id, end_id = id_ranges[file_name]
            print(file_name)

            with open(xhtml_file, 'r', encoding='utf-8') as file:
                content = file.read()

            parsed_html = BeautifulSoup(content, 'lxml')
            capture_text = False
            extracted_text = ""

            for div in parsed_html.find_all('div', id=True):
                if div.get('id') == start_id:
                    capture_text = True
                if capture_text:
                    extracted_text += div.get_text(separator="\n") + "\n\n"
                    # print(extracted_text)
                if div.get('id') == end_id:
                    capture_text = False  
            # Salvesta tekst txt faili
            output_file_path = xhtml_file.replace('.xhtml', '.txt').replace('aruandedXHTML','aastaaruanded')
            with open(output_file_path, 'w', encoding='utf-8') as output_file:
                output_file.write(extracted_text)
            print(f"Text from {file_name}.xhtml between {start_id} and {end_id} has been saved to {output_file_path}")


In [15]:
aruandedXHTML = glob.glob('aruanded/aruandedXHTML/*')

extract_text_from_xhtml(aruandedXHTML, xhtml_nums)

ako
Text from ako.xhtml between pf57 and pf38_1 has been saved to aruanded/aastaaruanded\rep_ako_2023.txt
apg
Text from apg.xhtml between pf1 and pf2c has been saved to aruanded/aastaaruanded\rep_apg_2023.txt
ign
Text from ign.xhtml between pfa3 and pf136 has been saved to aruanded/aastaaruanded\rep_ign_2023.txt
kne1


c:\Users\brandon\AppData\Local\anaconda3\envs\estnltk_env\Lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


Text from kne1.xhtml between pfa3 and pf100 has been saved to aruanded/aastaaruanded\rep_kne1_2023.txt
pzv
Text from pzv.xhtml between pf57 and pf70 has been saved to aruanded/aastaaruanded\rep_pzv_2023.txt
rsu
Text from rsu.xhtml between pf1 and pf34 has been saved to aruanded/aastaaruanded\rep_rsu_2023.txt
sab
Text from sab.xhtml between pf1 and pf4c has been saved to aruanded/aastaaruanded\rep_sab_2023.txt
vlp
Text from vlp.xhtml between pf1 and pf4b has been saved to aruanded/aastaaruanded\rep_vlp_2023.txt


In [14]:

'''Funktsioon loomaks sisukordade sõnastik'''
def create_toc_dict(paths):
    toc_dict = {}
    for path in paths:
        #print(path)
        file_name = path.split('\\')[-1].split('.')[0].split('_')[1]
        doc = fitz.open(path)
        
        toc_patterns = ["sisukord", "table of contents", "content", "contents"]

        for page_num in range(len(doc)):
            # print('PAGE NUMBER:',page_num+1)
            text_lines = doc[page_num].get_text().splitlines()
            for line in text_lines:
                # print('RIDA:',line)
                line = line.strip().lower() 
                for pattern in toc_patterns:
                    if line == pattern.lower():
                        toc_dict[file_name] = page_num
                        break  # Leiame esimese vastavuse ja katkestame tsükli

            if file_name in toc_dict:  # Kui oleme lehekülje leidnud, ei ole vaja edasi otsida
                break
    return toc_dict

In [15]:
aastaaruanded = glob.glob('aruanded/aastaaruanded/*.pdf')
toc = create_toc_dict(aastaaruanded)
print(aastaaruanded[3:4])
print('sisukorra lk nr:',toc)
print(len(toc))

['aruanded/aastaaruanded\\rep_dgr_2022.pdf']
sisukorra lk nr: {'arc': 2, 'aug': 1, 'cpa': 2, 'dgr': 1, 'eeg': 2, 'eft': 2, 'egr': 1, 'hae': 2, 'hpr': 2, 'inf': 2, 'lhv': 3, 'mrk': 1, 'ncn': 2, 'ntu': 1, 'pkg': 2, 'prf': 2, 'saf': 1, 'sfg': 1, 'tal': 1, 'tel1': 1, 'tkm': 2, 'tsm': 2, 'tve': 1}
23


In [16]:
print('sisukorra lk nr:',toc)
print(len(toc))

sisukorra lk nr: {'arc': 2, 'aug': 1, 'cpa': 2, 'dgr': 1, 'eeg': 2, 'eft': 2, 'egr': 1, 'hae': 2, 'hpr': 2, 'inf': 2, 'lhv': 3, 'mrk': 1, 'ncn': 2, 'ntu': 1, 'pkg': 2, 'prf': 2, 'saf': 1, 'sfg': 1, 'tal': 1, 'tel1': 1, 'tkm': 2, 'tsm': 2, 'tve': 1}
23


In [17]:
def trim_pdf_before_keyword_section(path, keywords, toc=None):
    doc = fitz.open(path)
    text_output_path = path.replace('.pdf', '.txt')
    trim_start_page = None
    file_name = path.split('\\')[-1].split('.')[0].split('_')[1]
    nbr = toc.get(file_name)
    # print(nbr)
    if nbr is not None:
        toc_text = doc[nbr].get_text().replace('\n',' ')
        # print(toc_text)
        for keyword in keywords:
            pattern = fr"{re.escape(keyword)}\s*(?:\.+\s*)+(\d+)"
            #pattern = f"{keyword}\\s*(\\d+)"
            # print('PATTERN',pattern)
            match = re.search(pattern, toc_text, re.IGNORECASE)
            #print(match)
            if match:
                trim_start_page = int(match.group(1)) - 1
                #trim_start_page = 99
                # print('LEITUD TEKST!!!!!!!!!!!:',trim_start_page)
                break  

    with open(text_output_path, "w", encoding="utf-8") as text_file:
        if trim_start_page is not None:
            for page_num in range(trim_start_page):
                page_text = doc[page_num].get_text()
                text_file.write(page_text)
        else:
            for page_num in range(len(doc)):
                page_text = doc[page_num].get_text()
                text_file.write(page_text)

    doc.close()
    # print(trim_start_page)
    return trim_start_page

    

In [18]:
aastaaruanded = glob.glob('aruanded/aastaaruanded/*.pdf')

keywords = ['Konsolideeritud raamatupidamise aastaaruanne',
'Kontserni raamatupidamise aastaaruanne',
'Konsolideerimisgrupi raamatupidamise aastaaruanne',
'RAAMATUPIDAMISE AASTAARUANNE',
'Consolidated and separate financial statements',
'Financial Statements']
cuts = {}
for aruanne in aastaaruanded:
    print()
    print('ARUANNE:',aruanne)
    file_name = aruanne.split('\\')[-1].split('.')[0].split('_')[1]
    cut = trim_pdf_before_keyword_section(aruanne,keywords,toc)
    print(cut)
    cuts[file_name] = cut


ARUANNE: aruanded/aastaaruanded\rep_arc_2023.pdf
28

ARUANNE: aruanded/aastaaruanded\rep_aug_2022.pdf
77

ARUANNE: aruanded/aastaaruanded\rep_cpa_2023.pdf
40

ARUANNE: aruanded/aastaaruanded\rep_dgr_2022.pdf
None

ARUANNE: aruanded/aastaaruanded\rep_eeg_2023.pdf
91

ARUANNE: aruanded/aastaaruanded\rep_eft_2023.pdf
18

ARUANNE: aruanded/aastaaruanded\rep_egr_2023.pdf
None

ARUANNE: aruanded/aastaaruanded\rep_hae_2023.pdf
None

ARUANNE: aruanded/aastaaruanded\rep_hpr_2023.pdf
57

ARUANNE: aruanded/aastaaruanded\rep_idx_2023.pdf
None

ARUANNE: aruanded/aastaaruanded\rep_inf_2022.pdf
None

ARUANNE: aruanded/aastaaruanded\rep_lhv_2023.pdf
None

ARUANNE: aruanded/aastaaruanded\rep_mrk_2023.pdf
41

ARUANNE: aruanded/aastaaruanded\rep_ncn_2023.pdf
58

ARUANNE: aruanded/aastaaruanded\rep_ntu_2022.pdf
None

ARUANNE: aruanded/aastaaruanded\rep_pkg_2023.pdf
52

ARUANNE: aruanded/aastaaruanded\rep_prf_2022.pdf
30

ARUANNE: aruanded/aastaaruanded\rep_ptr_2023.pdf
None

ARUANNE: aruanded/aastaaruand

In [19]:
def trim_problematic_pdf(path,cut):
    doc = fitz.open(path)
    text_output_path = path.replace('.pdf', '.txt')

    with open(text_output_path, "w", encoding="utf-8") as text_file:
        for page_num in range(cut):
            page_text = doc[page_num].get_text()
            text_file.write(page_text)

    doc.close()
    # print(trim_start_page)
    return text_output_path

In [20]:
print(cuts)
print(len(cuts))
problematic_cuts = []
for firm,cut in cuts.items():
    if cut is None:
        problematic_cuts.append(firm)
problematic_cuts.remove('dgr') # eemaldame, sest sellel esg raport olemas
problematic_cuts.remove('idx') # eemaldame, sest selle pdf vigane
problematic_cuts.remove('ptr') # eemaldame, sest sellel esg raport olemas
print(problematic_cuts)
problematic_reports = {
    'egr':89, 
    'hae':87, 
    'inf':14, 
    'lhv':81, 
    'ntu':41, 
    'saf':5, 
    'tel1':134, 
    'tsm':75
}

aastaaruanded = glob.glob('aruanded/aastaaruanded/*.pdf')

for aruanne in aastaaruanded:
    for firm,cut in problematic_reports.items():
        if firm in aruanne:
            trim_problematic_pdf(aruanne,cut)
    

{'arc': 28, 'aug': 77, 'cpa': 40, 'dgr': None, 'eeg': 91, 'eft': 18, 'egr': None, 'hae': None, 'hpr': 57, 'idx': None, 'inf': None, 'lhv': None, 'mrk': 41, 'ncn': 58, 'ntu': None, 'pkg': 52, 'prf': 30, 'ptr': None, 'saf': None, 'sfg': 25, 'tal': 61, 'tel1': None, 'tkm': 50, 'tsm': None, 'tve': 107}
25
['egr', 'hae', 'inf', 'lhv', 'ntu', 'saf', 'tel1', 'tsm']


In [21]:
def clean_and_structure_text(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()

    text = text.strip()
    
    # Eemaldame üleliigsed tühikud
    text = re.sub(r'\s+', ' ', text)

    # Liidame tükeldatud sõnad
    text = re.sub(r'-\s+', '', text)

    # Eemaldame mitteolulised sümbolid gpt mudeli jaoks
    # text = re.sub(r'[^a-zA-Z0-9.,;:!?()"\']+', ' ', text)

    # Kirjutame faili sisu üle
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(text)
        

In [22]:
aastaaruanded = glob.glob('aruanded/aastaaruanded/*.txt')

for aruanne in aastaaruanded:
    clean_and_structure_text(aruanne)

In [23]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

txt_files = glob.glob('aruanded/aastaaruanded/*.txt')
token_counts = {}


for file_path in txt_files:
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    
    cleaned_text = text
    file_name = file_path.split('/')[-1]

    tokens = tokenizer.tokenize(cleaned_text)
    
    token_counts[file_name] = len(tokens)

print(f"Tokenite arv: {token_counts}")

c:\Users\brandon\AppData\Local\anaconda3\envs\estnltk_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


Tokenite arv: {'aastaaruanded\\ako.txt': 108829, 'aastaaruanded\\apg.txt': 86013, 'aastaaruanded\\ign.txt': 121706, 'aastaaruanded\\kne1.txt': 34161, 'aastaaruanded\\pzv.txt': 4495, 'aastaaruanded\\rep_arc_2023.txt': 24883, 'aastaaruanded\\rep_aug_2022.txt': 40251, 'aastaaruanded\\rep_cpa_2023.txt': 44832, 'aastaaruanded\\rep_dgr_2022.txt': 45928, 'aastaaruanded\\rep_eeg_2023.txt': 102229, 'aastaaruanded\\rep_eft_2023.txt': 26620, 'aastaaruanded\\rep_egr_2023.txt': 108010, 'aastaaruanded\\rep_hae_2023.txt': 85999, 'aastaaruanded\\rep_hpr_2023.txt': 41715, 'aastaaruanded\\rep_idx_2023.txt': 6323, 'aastaaruanded\\rep_inf_2022.txt': 16119, 'aastaaruanded\\rep_lhv_2023.txt': 115726, 'aastaaruanded\\rep_mrk_2023.txt': 66892, 'aastaaruanded\\rep_ncn_2023.txt': 79380, 'aastaaruanded\\rep_ntu_2022.txt': 25903, 'aastaaruanded\\rep_pkg_2023.txt': 54259, 'aastaaruanded\\rep_prf_2022.txt': 33854, 'aastaaruanded\\rep_ptr_2023.txt': 5208, 'aastaaruanded\\rep_saf_2023.txt': 2803, 'aastaaruanded\\rep_

In [24]:
sorted_token_counts = sorted(token_counts.items(), key=lambda x: x[1], reverse=True)
for file_name, token_count in sorted_token_counts:
    print(f"{file_name}: {token_count}")

aastaaruanded\ign.txt: 121706
aastaaruanded\rep_lhv_2023.txt: 115726
aastaaruanded\ako.txt: 108829
aastaaruanded\rep_egr_2023.txt: 108010
aastaaruanded\rep_eeg_2023.txt: 102229
aastaaruanded\rep_tve_2023.txt: 100804
aastaaruanded\apg.txt: 86013
aastaaruanded\rep_hae_2023.txt: 85999
aastaaruanded\rep_ncn_2023.txt: 79380
aastaaruanded\rep_tkm_2023.txt: 74629
aastaaruanded\sab.txt: 73580
aastaaruanded\rep_mrk_2023.txt: 66892
aastaaruanded\rep_tel1_2023.txt: 63707
aastaaruanded\rep_tsm_2023.txt: 63000
aastaaruanded\rep_tal_2023.txt: 55169
aastaaruanded\rep_pkg_2023.txt: 54259
aastaaruanded\vlp.txt: 51833
aastaaruanded\rep_dgr_2022.txt: 45928
aastaaruanded\rep_cpa_2023.txt: 44832
aastaaruanded\rep_hpr_2023.txt: 41715
aastaaruanded\rep_aug_2022.txt: 40251
aastaaruanded\rep_sfg_2022.txt: 37407
aastaaruanded\kne1.txt: 34161
aastaaruanded\rep_prf_2022.txt: 33854
aastaaruanded\rsu.txt: 29180
aastaaruanded\rep_eft_2023.txt: 26620
aastaaruanded\rep_ntu_2022.txt: 25903
aastaaruanded\rep_arc_2023.tx

In [ ]:
esg_data_path = os.getenv('ESG_INPUT')
esg_data = pd.read_excel(esg_data_path)
grouped_data = esg_data.groupby(['Topic', 'Question'])['Answers'].apply(list).reset_index()
order_of_topics = [
    'Stakeholder engagement and reporting',
    'Leadership commitment', 
    'Impact assesment', 
    'Planning', 
    'Execution', 
    'Monitoring', 
    'Performance improvement', 
    'Across all topics'
]

grouped_data['Topic'] = pd.Categorical(grouped_data['Topic'], categories=order_of_topics, ordered=True)
sorted_data = grouped_data.sort_values('Topic')
sorted_data

In [ ]:
esg_val_path = os.getenv('ESG_VAL')
esg_val = pd.read_excel(esg_val_path)
esg_val

In [4]:
def get_answer_text(row, answer_column):
    if pd.notna(row[answer_column]):
        answer_number = int(row[answer_column]) - 1 
        if answer_number >= 0 and answer_number < len(row['Answers']):
            return row['Answers'][answer_number]
    return None

In [ ]:
merged_data = esg_val.merge(sorted_data, on='Question', how='left')
merged_data['SS Explained'] = None
merged_data['GPT Explained'] = None
column_order = ['Company', 'Abbreviation','Topic', 'Question', 'SS Answer', 'SS Explained', 'GPT ANSWER 1','GPT Explained',  'Answers']
merged_data = merged_data[column_order]

merged_data['SS Explained'] = merged_data.apply(lambda row: get_answer_text(row, 'SS Answer'), axis=1)

merged_data[:10]

In [ ]:
ako_data = merged_data[merged_data['Abbreviation'] == 'AKO']
ako_data  

In [98]:
import random

txt_files = glob.glob('aruanded/aastaaruanded/*ako.txt')

for file in txt_files:
    file_name = os.path.basename(file.replace('\\', '/')) 
    abbrev = file_name.split('.')[0]  
    with open(file, 'r', encoding='utf-8') as file:
        user_input_text = file.read()
    print("Abbreviation:", abbrev.upper())
    for index, row in merged_data[merged_data['Abbreviation'] == abbrev.upper()].iterrows():
        topic = row['Topic']
        question = row['Question']
        answers = [f"{a}" for a in row['Answers']] 
        # print('ANSWERS!!!-------',answers)
        system_prompt = f"Act as a sustainability specialist, who is filling in the pre-analysis questions based on the SASB methodology. " \
                    f"Firstly, focus to find the relevant parts of text based on the topic and then try to answer. " \
                    f"Choose one of the answer options, which is most suitable according to the provided report and give only the answer order number." \
                    f"Give only one most accurate answer.\n\n" \
                    f"Topic: {topic}\n\n" \
                    f"Question: {question}\n\n" \
                    f"Answers:\n" + "\n".join(answers)
        system_prompts = [
            {
                "role": "system",
                "content": system_prompt
                },
            {
                "role": "user",
                "content": user_input_text
                }
            ]
        
        #print("Request data:")
        #for message in system_prompts:
            #print(f"{message['role']}: {message['content']}\n")
        # client = openai.OpenAI(api_key=os.getenv('MY_API_KEY'))
        
        # chat_completion = client.chat.completions.create(
        #     messages=system_prompts,
        #     model="gpt-4-turbo",  
        #     )

        # contents = [choice['message']['content'] for choice in chat_completion['choices'] if choice['message']['role'] == 'assistant']
        # merged_data.loc[index, 'GPT ANSWER 1'] = contents
        answer = random.randint(1, 3)
        merged_data.loc[index, 'GPT ANSWER 1'] = answer

        

Abbreviation: AKO


In [103]:
chat_completion = {
    'id': 'chatcmpl-9Gm2yzIdtAtyKTKoqVpv7OpVNlXLV',
    'choices': [
        {
            'finish_reason': 'stop',
            'index': 0,
            'logprobs': None,
            'message': {
                'content': "First response.",
                'role': 'assistant',
                'function_call': None,
                'tool_calls': None
            }
        },
        {
            'finish_reason': 'stop',
            'index': 1,
            'logprobs': None,
            'message': {
                'content': "Second response.",
                'role': 'system',
                'function_call': None,
                'tool_calls': None
            }
        },
        {
            'finish_reason': 'stop',
            'index': 2,
            'logprobs': None,
            'message': {
                'content': "Third response.",
                'role': 'assistant',
                'function_call': None,
                'tool_calls': None
            }
        }
    ],
    'created': 1713784968,
    'model': 'gpt-4-0125-preview',
    'object': 'chat.completion',
    'system_fingerprint': 'fp_122114e45f',
    'usage': {
        'completion_tokens': 17,
        'prompt_tokens': 111067,
        'total_tokens': 111084
    }
}


contents = [choice['message']['content'] for choice in chat_completion['choices'] if choice['message']['role'] == 'assistant']

print(contents)

['First response.', 'Third response.']


In [4]:
# INPUT: report
with open('aruanded/aastaaruanded/ako.txt', 'r', encoding='utf-8') as file:
    user_input_text = file.read()

for _, row in sorted_data.iterrows():
    topic = row['Topic']
    question = row['Question']
    answers = [f"{i+1}. {a.strip()}" for i, a in enumerate(row['Answers'])] 
    # print('ANSWERS!!!-------',answers)
    system_prompt = f"Act as a sustainability specialist, who is filling in the pre-analysis questions based on the SASB methodology. " \
                    f"Firstly, focus to find the relevant parts from text based on the topic and then try to answer. " \
                    f"Choose one of the answer options, which is most suitable according to the provided report and give only the answer order number." \
                    f"Give only one most accurate answer.\n\n" \
                    f"Topic: {topic}\n\n" \
                    f"Question: {question}\n\n" \
                    f"Answers:\n" + "\n".join(answers)

    system_prompts = [
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": user_input_text
        }
    ]

client = openai.OpenAI(api_key=os.getenv('MY_API_KEY'))

chat_completion = client.chat.completions.create(
    messages=system_prompts,
    model="gpt-4-turbo",  
)

responses = []
for message in chat_completion['choices'][0]['message']:
    if message['role'] == 'assistant':
        responses.append(message['content'])

# Assuming each response is self-contained, create a DataFrame
df = pd.DataFrame(responses, columns=['Responses'])

print(df)

ChatCompletion(id='chatcmpl-9Gm2yzIdtAtyKTKoqVpv7OpVNlXLV', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="I'm sorry, but I cannot provide the answers to the disclosure questions as requested.", role='assistant', function_call=None, tool_calls=None))], created=1713784968, model='gpt-4-0125-preview', object='chat.completion', system_fingerprint='fp_122114e45f', usage=CompletionUsage(completion_tokens=17, prompt_tokens=111067, total_tokens=111084))


In [11]:
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def completion_with_backoff(**kwargs):
    """
    Calls the OpenAI API with exponential backoff on retries. It will retry up to 6 times with increasing wait times.
    """
    return openai.ChatCompletion.create(**kwargs)

#https://cookbook.openai.com/examples/how_to_handle_rate_limits

In [17]:
from openai import OpenAI

client = openai.OpenAI(api_key=os.getenv('MY_API_KEY'))

stream = client.chat.completions.create(
    model="gpt-4",
    messages=[{"role": "user", "content": "Say this is a test"}],
    stream=True,
)

responses = []
for message in stream['choices'][0]['message']:
    if message['role'] == 'assistant':
        responses.append(message['content'])

df = pd.DataFrame(responses, columns=['Responses'])

print(df)


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}